# Dapr in Action: From Core Concepts to AI Agents

Welcome to the "Dapr in Action: From Core Concepts to AI Agents" workshop!

This workshop provides a hands-on introduction to Dapr Agents through simple examples. By the end of this workshop, you will be able to:

* Create and interact with LLMs using Dapr Agents
* Build custom tools for agents to use
* Implement the ReAct pattern (Reasoning + Action)
* Create stateful workflows with multiple LLM steps

### Notebook 1. Basic LLM Usage & Understanding Response Objects

This notebook demonstrates how to use the `HFHubChatClient` in [dapr-agents](https://github.com/dapr/dapr-agents/tree/main) for basic tasks. We will explore how to:

* Initialize the Hugging Face Chat client.
* Generate responses to simple prompts.
* Use a `.prompty` file to provide context/history for enhanced generation.


#### **[Only if you are using Google Colab]** Setting up dependencies

First, let's install all the dependencies. Execute the cell below.

In [ ]:
!pip install dapr-agents==0.5.1
!pip install yfinance>=0.2.61
!pip install prompty

#### **[Only if you are using Google Colab]** Setting up secrets

Next, we need to set the secrets. In Google Colab, in the left-side menu navigate to Secrets->Add new secret.

Create two secrets with Notebook access called: `HUGGINGFACE_API_KEY` and `CURRENTS_API_KEY`.

You can find their values [here](https://privatebin.net/?ec00cd7f2c464e55#8xMHSFNHyTrGe8f1DhyB62qCmb9RwrPoFExV5pWKXZEs). The password will be shared during the workshop. You can also create your own free keys here (it requires creating an account):

- https://currentsapi.services/en
- https://huggingface.co/

Then execute the cell below:

In [ ]:
import os

from google.colab import userdata

os.environ["HUGGINGFACE_API_KEY"] = userdata.get('HUGGINGFACE_API_KEY')
os.environ["CURRENTS_API_KEY"] = userdata.get('CURRENTS_API_KEY')

#### **[Only if you are using Jupyter Notebooks]** Setting up secrets

Next, we need to set the secrets. You can find their values [here](https://privatebin.net/?ec00cd7f2c464e55#8xMHSFNHyTrGe8f1DhyB62qCmb9RwrPoFExV5pWKXZEs). The password will be shared during the workshop. You can also create your own free keys here (it requires creating an account):

- https://currentsapi.services/en
- https://huggingface.co/

Set the secrets values in `helper_secrets.py`. Then execute the cell below:

In [1]:
import os

import helper_secrets

os.environ["HUGGINGFACE_API_KEY"] = helper_secrets.HUGGINGFACE_API_KEY
os.environ["CURRENTS_API_KEY"] = helper_secrets.CURRENTS_API_KEY

#### Exercise 1 (a): Setup and call the HuggingFace Client using Dapr Agents

Run the following piece of code:

In [2]:
from dapr_agents import HFHubChatClient
from dapr_agents.types import UserMessage
from prompty import Prompty

llm = HFHubChatClient(
    api_key=os.getenv("HUGGINGFACE_API_KEY"),
    model='microsoft/Phi-3-mini-4k-instruct'
    )

response = llm.generate(
    messages=[UserMessage("What is the impact of AI on the news? Limit the output to 1 sentance.")]
)

/Users/danaarsovska/.pyenv/versions/3.11.4/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


**Expected output:** The LLM will respond with an explanation.

Next, let's analyze the response object:

In [3]:
print("Full response object:", response)
print("Content:", response.get_content())
print("Model used:", response.model)
print("Tokens used:", response.usage.get("total_tokens"))

Full response object: choices=[Choice(finish_reason='stop', index=0, message=MessageContent(content='AI plays a significant role in news production, from automating news writing to aiding in fact-checking and content curation, impacting the efficiency, accessibility, and potentially the objectivity of the news.', role='assistant'), logprobs=None)] created=1747397316 id='' model='microsoft/Phi-3-mini-4k-instruct' object='chat.completion' usage={'completion_tokens': 46, 'prompt_tokens': 24, 'total_tokens': 70}
Content: AI plays a significant role in news production, from automating news writing to aiding in fact-checking and content curation, impacting the efficiency, accessibility, and potentially the objectivity of the news.
Model used: microsoft/Phi-3-mini-4k-instruct
Tokens used: 70


#### Exercise 1 (b): Asking for current information

Now let's try tweaking the prompt:

In [4]:
response = llm.generate(
    messages=[UserMessage("Get me the news today.")]
)

In [5]:
print(response.get_content())

I'm sorry, but I can't access real-time information or browse the internet. I'm designed to work with a wide array of information up until my last update in early 2023. For the latest news, I recommend checking a reliable news source like BBC News, CNN, The New York Times, or directly accessing news apps or websites according to your preference.


**Expected output:** The LLM will not be able to fetch the latest news. Why is that?

#### Exercise 1 (c): Prompt engineering

Try different prompts to understand prompt design:

* Request a list
* Return a single word
* Ask for a creative story

In [ ]:
response = llm.generate(
    # TODO: insert your code here
    messages=[UserMessage("")]
)

In [ ]:
print(response.get_content())

#### [Optional] Exercise 1 (d): Parameter tweaking

You can also try experimenting with model parameters, for example:

* set the temperature (A float value to control the temperature of the model. Used to optimize for consistency and creativity)
* Optional: Re-initialze the client with a different model: https://huggingface.co/models

In [ ]:
response = llm.generate(
    messages=[UserMessage("What is the impact of AI on the news?")]
    # TODO: insert your code here
)

**Discussion**: What makes a good prompt? How do parameters affect output?

#### [Optional] Exercise 1 (e): Chat completion using a prompty file for context

In Dapr Agents, we can use prompty to provide additional context/history to our LLM. Before running the cells below, nalyze its contents of `prompt_text`.

In [6]:
prompt_text = """
---
name: Basic Prompt
description: A basic prompt that uses the chat API to answer questions
model:
    api: chat
    configuration:
        type: huggingface
        name: microsoft/Phi-3-mini-4k-instruct
    parameters:
        max_tokens: 128
        temperature: 0.2
inputs:
  question:
    type: string
sample:
  "question": "Who is the most famous person in the world?"
---
system:
You are an AI assistant who helps people find information.
As the assistant, you answer questions briefly, succinctly.

user:
{{question}}
"""

In [7]:
llm = HFHubChatClient.from_prompty(prompt_text)
print(llm.generate(input_data={"question":"What is your name?"}))

choices=[Choice(finish_reason='length', index=0, message=MessageContent(content="I'm Phi and my purpose as Microsoft language model GPT-3 developed by MS research team led to assist users in various tasks involving natural languages processing like text generation or translation among others while maintaining user privacy at all times without personal data storage capabilities beyond this interaction session for security reasons according with our policy guidelines on ethical use of technology which includes respectful communication practices that promote positive interactions between humans across diverse backgrounds irrespective their race gender age religion sexual orientation nationality etc., ensuring equal opportunities accessibility inclusivity diversified representation equitable distribution benefits fairness transparency accountability integrity hon", role='assistant'), logprobs=None)] created=1747397342 id='' model='microsoft/Phi-3-mini-4k-instruct' object='chat.completion' 

Change the system instructions to make the assistant more formal, humorous, or detailed. Then re-execute the cells.

#### Troubleshooting

1. **API Key Issues**: If you see an authentication error, verify your HuggingFace API key in the `helper_secrets.py` file
2. **Python Version**: If you encounter compatibility issues, make sure you're using Python 3.10+
3. **Environment Activation**: Ensure your virtual environment is activated before running examples
4. **Import Errors**: If you see module not found errors, verify that `pip install -r requirements.txt` completed successfully